In [23]:
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
!dir data-labeled

 Volume in drive D is New Volume
 Volume Serial Number is CC69-686C

 Directory of D:\Projects\1-telegram\data-labeled

14-11-2021  13:54    <DIR>          .
15-11-2021  17:39    <DIR>          ..
13-11-2021  15:32           496,846 combined-super-clean-data.xlsx
29-10-2021  00:22            43,643 CryptoRobinhooders_chat_data_clean__.xlsx
29-10-2021  02:22           193,083 Satoshi_club_chat_data_clean__.xlsx
13-11-2021  12:53         2,049,592 telegram_data_8th_NOV.csv
               4 File(s)      2,783,164 bytes
               2 Dir(s)  277,038,145,536 bytes free


In [37]:
robin = pd.read_excel(r'data-labeled/CryptoRobinhooders_chat_data_clean__.xlsx')
robin = robin.loc[:, ['Raw Data', 'Label']]

satoshi = pd.read_excel(r'data-labeled/Satoshi_club_chat_data_clean__.xlsx')
satoshi = satoshi.loc[:, ['Raw Data', 'Label']]

telegram_data = pd.read_csv('data-labeled/telegram_data_8th_NOV.csv', encoding='ISO-8859-1')
telegram_data = telegram_data.loc[:, ['text', 'Unnamed: 5']]
telegram_data.columns = ['Raw Data', 'Label']
telegram_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [38]:
robin.Label.value_counts()

Neutral     133
positive     70
negative      7
Name: Label, dtype: int64

In [39]:
satoshi.Label.value_counts()

positive     718
neutral      383
Neutral      185
negative     153
Positive     145
Negative      74
positive       7
 neutral       1
Name: Label, dtype: int64

In [40]:
telegram_data.Label.value_counts()

irrelevant    4049
positive      3458
neutral       3326
negative       710
Irrelevant     242
Positive       218
Neutral         84
Negative        12
Irrlevant       11
irrlevant        4
nue              4
posiitve         2
ne               1
Postive          1
Postiive         1
v                1
Name: Label, dtype: int64

In [41]:
satoshi['Label'] = satoshi['Label'].str.strip().str.lower()
satoshi.fillna('other', inplace=True)

In [42]:
robin['Label'] = robin['Label'].str.strip().str.lower()
robin.fillna('other', inplace=True)

In [43]:
telegram_data['Label'] = telegram_data['Label'].str.strip().str.lower()
telegram_data.fillna('other', inplace=True)
telegram_data['Label'] = telegram_data.Label.replace('nue', 'neutral')
telegram_data['Label'] = telegram_data.Label.replace('positiive', 'positive')
telegram_data['Label'] = telegram_data.Label.replace('irrlevant', 'irrelevant')
telegram_data['Label'] = telegram_data.Label.replace('postive', 'positive')
telegram_data['Label'] = telegram_data.Label.replace('postiive', 'positive')
telegram_data['Label'] = telegram_data.Label.replace('posiitve', 'positive')
telegram_data['Label'] = telegram_data.Label.replace('ne', 'neutral')
telegram_data = telegram_data[telegram_data['Label'] != 'v']
telegram_data['Label'] = telegram_data.Label.replace('irrelevant', 'other')

In [44]:
robin.columns, satoshi.columns, telegram_data.columns

(Index(['Raw Data', 'Label'], dtype='object'),
 Index(['Raw Data', 'Label'], dtype='object'),
 Index(['Raw Data', 'Label'], dtype='object'))

In [45]:
robin.shape, satoshi.shape, telegram_data.shape

((377, 2), (2832, 2), (12123, 2))

In [14]:
df = pd.concat([robin, satoshi, telegram_data], axis=0)
df.columns = ['raw_data', 'label']
df.reset_index(inplace=True, drop=True)
df.shape

(15332, 2)

In [24]:
df.label.value_counts()

other       5639
positive    4620
neutral     4117
negative     956
Name: label, dtype: int64

In [21]:
from plotly import express as px
%matplotlib inline

In [22]:
fig = px.histogram(df, x="label")
fig.update_layout(title_text='Label Count')
fig.show()

In [15]:
df.head()

,raw_data,label
0,**Welcome Sonika to CryptoRobinhooders🚀. \n\nF...,neutral
1,what is this mate?,neutral
2,What do you guys think about cryptoeats,neutral
3,#StripFinance - Enables #NFT owners to borrow ...,other
4,**JUST IN: **Facebook has chosen Coinbase as i...,other


In [27]:
# import demoji
import emoji

In [28]:
def clean_data(df):

    df['clean'] = df['raw_data'].apply(lambda x: str(x).replace('\n', ' '))
    df['clean'] = df['clean'].str.replace(pat='(@\w+)', repl='', regex=True)
    df['clean'] = df['clean'].replace(to_replace=' +', value=' ', regex=True)
    
    df['clean'] = df['raw_data'].astype(str).apply(lambda x: ' '.join([y for y in x.split() if len(y)<15]))
    df['clean'] = df['clean'].astype(str).str.replace(r"""[#*+\\\(\)_!:\-\[\]<>']""", '', regex=True)
    df['clean'] = df['clean'].astype(str).str.replace(r"""(\.{2,})""", '.', regex=True)
    df['clean'] = df['clean'].astype(str).str.replace(r"""[-a-zA-Z0-9@:%_\+.~#?&//=]{2,256}\.[a-z]{2,4}\b(\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?""", '<<<URL>>>', regex=True)
    df['clean'] = df['clean'].astype(str).str.replace(r""" +""", ' ', regex=True)
    df['clean'] = df['clean'].astype(str).str.replace(r"""\$(\d+\.?,?)+""", '<<<AMOUNT>>>', regex=True)
    df['clean'] = df['clean'].astype(str).str.replace(r"""(\d+%)""", '<<<PERCENTAGE>>>', regex=True)
    df['clean'] = df['clean'].apply(lambda x: emoji.demojize(x, delimiters=(" ___", "___ ")))
    df['sent_len'] = df['clean'].apply(lambda x: len(str(x).split()))
    df = df[df['sent_len'] > 2]
    df['clean'] = df['clean'].str.lower()
    df = df.loc[:, ['raw_data', 'clean', 'label']]
    
    return df

In [29]:
df = clean_data(df)
df.loc[:,['clean', 'label']].to_excel(r'data-labeled/combined-super-clean-data.xlsx', index=False)

C:\Users\itsni\AppData\Local\Temp/ipykernel_5972/3057033853.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
telegram_data.columns = ['raw_data', 'label']
telegram_data = clean_data(telegram_data)
telegram_data.loc[:,['clean', 'label']].to_excel(r'data-labeled/telegram-data-clean.xlsx', index=False)

C:\Users\itsni\AppData\Local\Temp/ipykernel_5972/3057033853.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
df.head()

,raw_data,clean,label
0,**Welcome Sonika to CryptoRobinhooders🚀. \n\nF...,welcome sonika to feel free to discuss and sha...,neutral
1,what is this mate?,what is this mate?,neutral
2,What do you guys think about cryptoeats,what do you guys think about cryptoeats,neutral
3,#StripFinance - Enables #NFT owners to borrow ...,stripfinance enables nft owners to borrow agai...,other
4,**JUST IN: **Facebook has chosen Coinbase as i...,just in facebook has chosen coinbase as its cu...,other
